In [3]:
import streamlit as st
import pandas as pd
import numpy as np
import altair as alt
import re
from datetime import datetime, timedelta
import os
import yaml
import helpers

In [4]:
def read_yaml_file(file_path):
    if os.path.isfile(file_path):
        # File exists, so read it
        try:
            settings = yaml.safe_load(file_path)
        except Exception as e:
            st.markdown(f"An error occurred while reading file {file_path}: {e}")
    else:
        st.markdown(f"The file {file_path} does not exist.")      
    return settings

In [8]:
settings = yaml.safe_load("inputs/settings/genx_settings.yml")

In [9]:
settings

'inputs/settings/genx_settings.yml'

In [15]:
import streamlit as st
import pandas as pd
import numpy as np
import altair as alt
import re
from datetime import datetime, timedelta
import os
import yaml



def read_power():
    file_path = "results/power.csv"  # Replace with your CSV file path
    df_power = pd.DataFrame()
    if os.path.isfile(file_path):
        # File exists, so read it
        try:
            df_power = pd.read_csv(file_path)
            df_power=df_power.iloc[2:]
            df_power.reset_index(drop=True, inplace=True)
            df_power = df_power.round(3)
        except Exception as e:
            st.markdown(f"An error occurred while reading the file: {e}")
    else:
        st.markdown(f"The file {file_path} does not exist.")   
    
    return df_power

def read_costs():
    file_path = "results\costs.csv"  # Replace with your CSV file path
    df_costs = pd.DataFrame()
    if os.path.isfile(file_path):
        # File exists, so read it
        try:
            df_costs = pd.read_csv(file_path)
            df_costs = df_costs.round(3)
        except Exception as e:
            st.markdown(f"An error occurred while reading the file: {e}")
    else:
        st.markdown(f"The file {file_path} does not exist.")   
    
    return df_costs



In [29]:
file_path = "results\Fuel_cost_plant.csv"  # Replace with your CSV file path
df_fuel = pd.read_csv(file_path)
df_fuel_total_cost = df_fuel.groupby('Fuel')['AnnualSumCosts'].sum().reset_index()

In [30]:
df_fuel_total_cost

,Fuel,AnnualSumCosts
0,Biomass,0.00000
1,CA_Natural_Gas,827.32188
2,Conventional_DR,0.00000
3,DefaultFuel,0.00000
4,Geothermal,0.00000


In [3]:
import pandas as pd
data = {
    'Resource': ['CAISO_Advanced_CCGT_candidate', 'CAISO_Aero_CT_candidate', 'CAISO_Reciprocating_Engine_candidate', 'CAISO_Shed_DR_Tranche1', 'CAM_CAM_RA_i'],
    'Revenue': [86.0206, 44.49509, 89.8247, 5.525751, 0.282928],
    'Cost': [744.027, 585.1162, 708.196, 262.8676, 7.804915]
}

df = pd.DataFrame(data)

# Round all numeric columns to 3 decimal places
df = df.round(3)
df

,Resource,Revenue,Cost
0,CAISO_Advanced_CCGT_candidate,86.021,744.027
1,CAISO_Aero_CT_candidate,44.495,585.116
2,CAISO_Reciprocating_Engine_candidate,89.825,708.196
3,CAISO_Shed_DR_Tranche1,5.526,262.868
4,CAM_CAM_RA_i,0.283,7.805


In [6]:
df_capacity['EndCap']['CAISO_Advanced_CCGT_candidate']

np.float64(0.00314)

In [ ]:

df_resources = pd.read_csv("inputs/resource_list.csv")

selected_type = "Thermal"

selected_zone = 1
resources_list = df_resources[(df_resources.Type == selected_type) & (df_resources.Zone == selected_zone)]

df_net_revenue = read_net_revenue().merge(resources_list, on="Resource", how='inner')


In [ ]:
df_net_revenue

In [ ]:

df_co2_emissions = read_co2_emissions()
df_co2_emissions_melt = df_co2_emissions.melt("Time", var_name="Zone", value_name="Co2 emission")
 

In [ ]:
df_co2_emissions_melt

In [ ]:
file_path_input = "inputs\\resources\\policy_assignments\\Resource_maximum_capacity_requirement.csv"  # Replace with your CSV file path
df_resources_maxcapreq_input = pd.read_csv(file_path_input)

df_resources_maxcapreq_input.set_index("Resource", inplace= True)
df_resources_maxcapreq_input_long = df_resources_maxcapreq_input.reset_index().melt(
    id_vars=['Resource'],
    var_name='Constraint',
    value_name='Constraint_Value'
)
df_resources_maxcapreq_input_long = df_resources_maxcapreq_input_long[df_resources_maxcapreq_input_long["Constraint_Value"]==1]
df_resources_maxcapreq_input_long.reset_index(inplace=True, drop=True)
df_resources_maxcapreq_input_long = df_resources_maxcapreq_input_long[["Resource","Constraint"]]
df_resources_maxcapreq_input_long['Constraint'] = df_resources_maxcapreq_input_long['Constraint'].str.replace("Max_Cap","MaxCapReq")

df_resources_maxcapreq_input_long = df_resources_maxcapreq_input_long.merge(policy_max_cap_req()[["Constraint",	"ConstraintDescription"]], how='left', on= "Constraint")
df_resources_maxcapreq_input_long = df_resources_maxcapreq_input_long[["Resource", "ConstraintDescription"]]
df_resources_maxcapreq_input_long.rename(columns={"ConstraintDescription": "Constraint"}, inplace=True)

In [ ]:
df_resources_maxcapreq_input_long

In [ ]:
def policy_max_cap_req():
    file_path_settings = "inputs\settings\genx_settings.yml"  
    df_maxcapreq_input = pd.DataFrame()
    df_maxcapreq_results = pd.DataFrame()
    
    # Open the file and load the YAML content
    with open(file_path_settings, 'r') as file:
        settings = yaml.safe_load(file)

    if 'MaxCapReq' in settings.keys():
        file_path_input = "inputs\policies\Maximum_capacity_requirement.csv"  # Replace with your CSV file path
        if os.path.isfile(file_path_input):
            # File exists, so read it
            try:
                df_maxcapreq_input = pd.read_csv(file_path_input)
                df_maxcapreq_input.iloc[:,0] = "MaxCapReq_"+df_maxcapreq_input.iloc[:,0].astype(str)
                df_maxcapreq_input.rename(columns={"MaxCapReqConstraint":"Constraint"}, inplace = True)
            except Exception as e:
                st.markdown(f"An error occurred while reading the file: {e}")

        file_path_results = "results/MaxCapReq_prices_and_penalties.csv"  # Replace with your CSV file path
        if os.path.isfile(file_path_results):
            # File exists, so read it
            try:
                df_maxcapreq_results = pd.read_csv(file_path_results)
                df_maxcapreq_input = df_maxcapreq_input.merge(df_maxcapreq_results, on='Constraint', how='left')
            except Exception as e:
                st.markdown(f"An error occurred while reading the file: {e}")
        else:
            st.markdown(f"The file {file_path_results} does not exist.")
        
        return df_maxcapreq_input.round(3)

In [ ]:
df_maxcapreq_input.round(3)

In [ ]:
policy_max_cap_req()

In [ ]:
file_path_settings = "inputs\settings\genx_settings.yml"  
df_maxcapreq_input = pd.DataFrame()
df_maxcapreq_results = pd.DataFrame()

# Open the file and load the YAML content
with open(file_path_settings, 'r') as file:
    settings = yaml.safe_load(file)

if 'MaxCapReq' in settings.keys():
    file_path_input = "inputs\policies\Maximum_capacity_requirement.csv"  # Replace with your CSV file path
    df_maxcapreq_input = pd.read_csv(file_path_input)
    df_maxcapreq_input.iloc[:,0] = "MaxCapReq_"+df_maxcapreq_input.iloc[:,0].astype(str)
    df_maxcapreq_input.rename(columns={"MaxCapReqConstraint":"Constraint"}, inplace = True)


    file_path_results = "results/MaxCapReq_prices_and_penalties.csv"  # Replace with your CSV file path
    df_maxcapreq_results = pd.read_csv(file_path_results)
    df_maxcapreq_input = df_maxcapreq_input.merge(df_maxcapreq_results, on='Constraint', how='left')

In [ ]:
df_maxcapreq_results

In [ ]:
df_maxcapreq_input

In [ ]:
file_path_results = "results/MaxCapReq_prices_and_penalties.csv"  # Replace with your CSV file path
df_maxcapreq_results = pd.read_csv(file_path_results)

In [ ]:
df_maxcapreq_results

In [ ]:
df_maxcapreq_input.iloc[:,0] = "MaxCapReg_"+df_maxcapreq_input.iloc[:,0].astype(str)
df_maxcapreq_input.rename(columns={"MaxCapReqConstraint":"Constraint"}, inplace=True)

In [ ]:
file_path_results = "results/MaxCapReq_prices_and_penalties.csv"  # Replace with your CSV file path
df_maxcapreq_results = pd.read_csv(file_path_results)

In [ ]:
df_maxcapreq_input.merge(df_maxcapreq_results, on='Constraint', how='left')

In [ ]:
df_maxcapreq_input

In [ ]:


    
    if os.path.isfile(file_path_results):
        # File exists, so read it
        try:
            
            df_maxcapreq_output = df_maxcapreq_output.round({'Price':3, 'Slack':3, 'Penalty':3})
        except Exception as e:
            st.markdown(f"An error occurred while reading the file: {e}")
    else:
        st.markdown(f"The file {file_path_results} does not exist.")

In [ ]:
dict_max_cap_req = dict()

file_path = "inputs\policies\Maximum_capacity_requirement.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_max_cap_req = pd.read_csv(file_path)
        df_hydro = df_hydro[["Resource","Zone"]]
        df_hydro["Type"] = "Hydro"
        df_resource = pd.concat([df_resource, df_hydro], axis=0)
    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

In [ ]:

# Creat resource list
df_resource = pd.DataFrame(columns=["Resource", "Type","Zone"])

file_path = "inputs/resources/Hydro.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_hydro = pd.read_csv(file_path)
        df_hydro = df_hydro[["Resource","Zone"]]
        df_hydro["Type"] = "Hydro"
        df_resource = pd.concat([df_resource, df_hydro], axis=0)
    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

file_path = "inputs/resources/Thermal.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_thermal = pd.read_csv(file_path)
        df_thermal = df_thermal[["Resource","Zone"]]
        df_thermal["Type"] = "Thermal"
        df_resource = pd.concat([df_resource, df_thermal], axis=0)
    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

file_path = "inputs/resources/Vre.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_vre = pd.read_csv(file_path)
        df_vre = df_vre[["Resource","Zone"]]
        df_vre["Type"] = "Vre"
        df_resource = pd.concat([df_resource, df_vre], axis=0)
    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

file_path = "inputs/resources/Storage.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_storage = pd.read_csv(file_path)
        df_storage = df_storage[["Resource","Zone"]]
        df_storage["Type"] = "Storage"
        df_resource = pd.concat([df_resource, df_storage], axis=0)

    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

file_path = "inputs/resources/Flex_demand.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_flex_demand = pd.read_csv(file_path)
        df_flex_demand = df_flex_demand[["Resource","Zone"]]
        df_flex_demand["Type"] = "Flex_demand"
        df_resource = pd.concat([df_resource, df_flex_demand], axis=0)
    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

file_path = "inputs/resources/Must_run.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_must_run = pd.read_csv(file_path)
        df_must_run = df_must_run[["Resource","Zone"]]
        df_must_run["Type"] = "Must_run"
        df_resource = pd.concat([df_resource, df_must_run], axis=0)
    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

file_path = "inputs/resources/Electrolyzer.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_electrolyzer = pd.read_csv(file_path)
        df_electrolyzer = df_electrolyzer[["Resource","Zone"]]
        df_electrolyzer["Type"] = "Electrolyzer"
        df_resource = pd.concat([df_resource, df_electrolyzer], axis=0)

    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

file_path = "inputs/resources/Vre_stor.csv"  # Replace with your CSV file path
if os.path.isfile(file_path):
    # File exists, so read it
    try:
        df_vre_stor = pd.read_csv(file_path)
        df_vre_stor = df_vre_stor[["Resource","Zone"]]
        df_vre_stor["Type"] = "Vre_stor"
        df_resource = pd.concat([df_resource, df_vre_stor], axis=0)

    except Exception as e:
        st.markdown(f"An error occurred while reading the file: {e}")

df_resource.reset_index(inplace=True, drop=True)
df_resource.to_csv("inputs/agg_resources.csv")

In [ ]:
df_co2_penalties = pd.read_csv("results/CO2_prices_and_penalties.csv")
df_esr_penalties = pd.read_csv("results/ESR_prices_and_penalties.csv")
df_maxcapreq_penalties = pd.read_csv("results/MaxCapReq_prices_and_penalties.csv")

df_co2_penalties.round({"CO2_Price":5})
df_esr_penalties.round({"ESR_Price":5, "ESR_AnnualSlack":5, "ESR_AnnualPenalty": 5})
df_maxcapreq_penalties.round({'Price':3, 'Slack':3, 'Penalty':3})


In [ ]:
df_maxcapreq_penalties

In [ ]:
df_maxcapreq_penalties